In [1]:
import os
os.chdir(r"E:\Car-Counting-and-tracking-in-highway-by-YOLOv5-main\Car-Counting-and-tracking-in-highway-by-YOLOv5-main")

In [74]:
import cv2
import torch
import numpy as np
from tracker import *


model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cap=cv2.VideoCapture('City Traffic (Stock Footage).mp4')

tracker = Tracker()

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 100.0, (1020,600))


def POINTS(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('FRAME')
cv2.setMouseCallback('FRAME', POINTS)
area = [(50,445),(470,445),(470,500),(0,500)]
area1 = [(560,445),(1000,445),(1010,500),(570,500)]

c = set()
C = set()
while True:
    ret,frame=cap.read()
    if not ret:
        break

    frame=cv2.resize(frame,(1020,600))
#     frame = cv2.flip(frame,0)
    cv2.polylines(frame,[np.array(area,np.int32)],True,(0,0,255),5)
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(0,255,0),5)
    results=model(frame)
    points = []
    for index , row in results.pandas().xyxy[0].iterrows():
        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])
        n=(row['name'])
        if 'car' in n:
            points.append([x1,y1,x2,y2])
            #cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),2)
            #cv2.putText(frame,str(n),(x1,y1),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
        
    boxes_id = tracker.update(points) 
    #print(boxes_id)
    for box_id in boxes_id:
        x , y , w , h , idd = box_id
        cv2.rectangle(frame,(x,y),(w,h),(255,0,0),3)
        cv2.putText(frame,str(idd),(x,y),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),3)
        cv2.circle(frame,(w,h),5,(20,20,255),-1)
        results = cv2.pointPolygonTest(np.array(area,np.int32),(w,h),False)
        results1 = cv2.pointPolygonTest(np.array(area1,np.int32),(w,h),False)
        #print(results)
        if results>= 0 :
            c.add(idd)
        if results1>= 0 :
            C.add(idd)    
        out.write(frame)
    a = len(c)
    b = len(C)
    cv2.putText(frame,'No. of Vehicles in the Red region is ='+str(a),(50,65),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),3)
    cv2.putText(frame,'No. of Vehicles in the Green region is ='+str(b),(50,100),cv2.FONT_HERSHEY_PLAIN,2,(0,255,0),3,-1)
    
    cv2.imshow("FRAME",frame)
    if cv2.waitKey(1)&0xFF==27:
        break
# cv2.VideoWriter('car.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 100.0, (100, 100), False)        
cap.release()
out.release()

cv2.destroyAllWindows()


Using cache found in C:\Users\ahmed/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\ahmed\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-3-8 Python-3.9.12 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


[459, 179]
[505, 145]
[508, 170]
[510, 171]
[539, 171]
